# Chapter10.SparkStreaming
Spark1.1不支持Python版本的。  
Spark2.4.5 现在全面支持Python版本了。 ------2020-3-10  
http://spark.apache.org/docs/latest/streaming-programming-guide.html#checkpointing

### 10.1 一个简单的例子

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
sc = SparkContext('local[*]', 'SparkStreaming')
ssc = StreamingContext(sc, 5)

In [3]:
lines = ssc.socketTextStream('localhost', 7777)

In [4]:
words = lines.flatMap(lambda line: line.split(' '))
pairs = words.map(lambda word: (word, 5))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)
wordCounts.pprint()

In [9]:
lines.foreachRDD(lambda x: print(x.collect()))

In [10]:
lines.pprint()

In [11]:
import os
lines.saveAsTextFiles('file://' + os.path.abspath('.') + '/streamDir/', 'txt')

In [15]:
ssc.start()  # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate
# 终端运行命令：nc -lk 7777
# 然后输入信息流

-------------------------------------------
Time: 2020-03-10 16:31:10
-------------------------------------------

[]
-------------------------------------------
Time: 2020-03-10 16:31:10
-------------------------------------------

-------------------------------------------
Time: 2020-03-10 16:31:15
-------------------------------------------

[]
-------------------------------------------
Time: 2020-03-10 16:31:15
-------------------------------------------

-------------------------------------------
Time: 2020-03-10 16:31:20
-------------------------------------------
('hellow', 5)
('world', 5)

['hellow world']
-------------------------------------------
Time: 2020-03-10 16:31:20
-------------------------------------------
hellow world

-------------------------------------------
Time: 2020-03-10 16:31:25
-------------------------------------------

[]
-------------------------------------------
Time: 2020-03-10 16:31:25
-------------------------------------------

--------------

KeyboardInterrupt: 

### 10.2 架构与抽象

### 10.3 转化操作

#### 无状态转化操作
DStream的无状态转化操作和普通RDD转化操作一样，只不过转化操作是作用于DStream内部的每个RDD上的。
- transform()操作   提供一个RDD到RDD的转化函数，会应用到DStream内部的每一个RDD上。相当于自创一个转化操作。

#### 有状态转化操作
- 滑动窗口 ： 以一个时间阶段为滑动窗口进行操作
- updateStateByKey() ： 用来跟踪每个键的状态变化

窗口操作:
- window():返回一个新的DStream表示窗口操作的结果数据。window生成DStream中的每个RDD会包含多个批次中的数据。

In [18]:
accessLogWindow = lines.window(10)
windowCounts = accessLogWindow.count()
print(windowCounts)

Py4JJavaError: An error occurred while calling o25.window.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:224)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:66)
	at org.apache.spark.streaming.dstream.WindowedDStream.<init>(WindowedDStream.scala:32)
	at org.apache.spark.streaming.dstream.DStream$$anonfun$window$1.apply(DStream.scala:767)
	at org.apache.spark.streaming.dstream.DStream$$anonfun$window$1.apply(DStream.scala:767)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.SparkContext.withScope(SparkContext.scala:699)
	at org.apache.spark.streaming.StreamingContext.withScope(StreamingContext.scala:264)
	at org.apache.spark.streaming.dstream.DStream.window(DStream.sca

-------------------------------------------
Time: 2020-03-10 16:40:00
-------------------------------------------

[]
-------------------------------------------
Time: 2020-03-10 16:40:00
-------------------------------------------

-------------------------------------------
Time: 2020-03-10 16:40:05
-------------------------------------------

[]
-------------------------------------------
Time: 2020-03-10 16:40:05
-------------------------------------------

-------------------------------------------
Time: 2020-03-10 16:40:10
-------------------------------------------

[]
-------------------------------------------
Time: 2020-03-10 16:40:10
-------------------------------------------

-------------------------------------------
Time: 2020-03-10 16:40:15
-------------------------------------------

[]
-------------------------------------------
Time: 2020-03-10 16:40:15
-------------------------------------------



### 10.4 输出操作
与RDD中的惰性求值类似，如果一个DStream没有执行输出操作，那么这些DStream就不会被求值。如果StreamingContext中没有设定输出操作，整个context就不会启动。
- saveAsTextFules()  将DStream存储到文件系统
- foreachRDD()   与transform类似  

### 10.5 输入源
- 文件流 textFileStream()
- Kafka
- Flume

In [22]:
data = ssc.textFileStream("./quickstart.txt")
data.pprint()
ssc.start()
ssc.awaitTermination()

Py4JJavaError: An error occurred while calling o23.textFileStream.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:224)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:66)
	at org.apache.spark.streaming.dstream.InputDStream.<init>(InputDStream.scala:45)
	at org.apache.spark.streaming.dstream.FileInputDStream.<init>(FileInputDStream.scala:82)
	at org.apache.spark.streaming.StreamingContext.fileStream(StreamingContext.scala:358)
	at org.apache.spark.streaming.StreamingContext$$anonfun$textFileStream$1.apply(StreamingContext.scala:414)
	at org.apache.spark.streaming.StreamingContext$$anonfun$textFileStream$1.apply(StreamingContext.scala:414)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.streaming.StreamingContext.withNamedScope(StreamingContext.scala:2

-------------------------------------------
Time: 2020-03-10 16:45:05
-------------------------------------------

[]
-------------------------------------------
Time: 2020-03-10 16:45:05
-------------------------------------------

-------------------------------------------
Time: 2020-03-10 16:45:10
-------------------------------------------

[]
-------------------------------------------
Time: 2020-03-10 16:45:10
-------------------------------------------

-------------------------------------------
Time: 2020-03-10 16:45:15
-------------------------------------------
('2', 5)
('1', 10)

['1', '1 2']
-------------------------------------------
Time: 2020-03-10 16:45:15
-------------------------------------------
1
1 2

-------------------------------------------
Time: 2020-03-10 16:45:20
-------------------------------------------
('1,', 5)
('2,3', 5)

['1, 2,3']
-------------------------------------------
Time: 2020-03-10 16:45:20
-------------------------------------------
1, 2,